In [ ]:
# # GPIO Purposes
# GPIO.setmode(GPIO.BCM)
# GPIO_ids = {
#     GPIO.IN: "IN",
#     GPIO.OUT: "OUT",
#     GPIO.SPI: "SPI",
#     GPIO.I2C: "I2C",
#     GPIO.HARD_PWM: "HARD_PWM",
#     GPIO.SERIAL: "SERIAL",
#     GPIO.UNKNOWN: "UNKNOWN"
# }
# 
# # Print out the active channels
# for pin in range(40):
#     function = GPIO.gpio_function(pin)
#     purpose = GPIO_ids.get(function, "UNKNOWN")
#     print(f"GPIO {pin} is {purpose}")
#     
# # GPIO.cleanup()
